In [145]:
## Fixed issue: we should consider only first innings
## Pending issue: 
#  -what is the best way to fix division by 0 problems
#  -tau has too many zeros!!


### prep

In [1]:
import pickle
import pandas as pd
import numpy as np
from scipy.stats import dirichlet, multinomial

In [2]:
with open('df1.pkl', 'rb') as f:
    data = pickle.load(f)
df = pd.DataFrame(data)
df.head(10)

,bowler,batsman,wicket,over,outcome
0,Mortaza_Bangladesh,Warner_Australia,0,1,0
1,Mortaza_Bangladesh,Warner_Australia,0,1,0
2,Mortaza_Bangladesh,Warner_Australia,0,1,1
3,Mortaza_Bangladesh,Watson_Australia,0,1,0
4,Mortaza_Bangladesh,Watson_Australia,0,1,1
5,Mortaza_Bangladesh,Warner_Australia,0,1,0
6,Shafiul_Bangladesh,Watson_Australia,0,2,0
7,Shafiul_Bangladesh,Watson_Australia,0,2,1
8,Shafiul_Bangladesh,Watson_Australia,0,2,0
9,Shafiul_Bangladesh,Watson_Australia,0,2,2


In [3]:
X = np.load("X_mat.npy")

In [4]:
n_mat = X.sum(axis=3, keepdims=True)

In [5]:
n_mat.shape

(374, 20, 10, 1)

In [6]:
emp_prob = X / (n_mat+1.e-15)
# emp_prob.shape
np.max(emp_prob)

0.9999999999999999

In [7]:
## Delta theorem multipliers
inv_prob = np.array([(1-p)/(p+1.e-15) for p in emp_prob])
inv_prob = inv_prob/(n_mat+1.e-15)
# inv_prob.shape

### alpha matrix

In [8]:
alpha=[]
v_inv=[]
for i in range(emp_prob.shape[0]):
    alpha_i=[]
    v_inv_i=[]
    for o in range(19):
        val = emp_prob[i,o+1]/(emp_prob[i,o]+1.e-15)
        alpha_i.append(val)
        v_i = np.square(val)*(inv_prob[i,o+1]+inv_prob[i,o]) 
        v_inv_i.append([1/(np.sqrt(x)+1.e-15) for x in v_i])
        # v_i_root = np.sqrt(v_i)
        # print(v_i_root.shape)
        # v_inv_i.append(v_i_root)
    alpha.append(alpha_i)
    v_inv.append(v_inv_i)
    
alpha = np.array(alpha)
v_inv = np.array(v_inv)
v_inv.shape

(374, 19, 10, 8)

In [9]:
den = np.sum(v_inv, axis=0)
num = np.sum(v_inv*alpha, axis=0)
o_trans = num/den

In [10]:
np.min(o_trans)

0.0

### beta matrix

In [11]:
beta=[]
w_inv=[]
for i in range(emp_prob.shape[0]):
    beta_o=[]
    y_inv_o=[]
    for o in range(20):
        beta_i=[]
        v_inv_i=[]
        for w in range(9):
            val = emp_prob[i,o,w+1]/(emp_prob[i,o,w]+1.e-15)
            beta_i.append(val)
            v_i = np.square(val)*(inv_prob[i,o,w+1]+inv_prob[i,o,w]) 
            v_inv_i.append([1/(np.sqrt(x)+1.e-15) for x in v_i])
            # print(val.shape)
        beta_o.append(beta_i)
        y_inv_o.append(v_inv_i)
    beta.append(beta_o)
    w_inv.append(y_inv_o)
    
beta = np.array(beta)
w_inv = np.array(w_inv)
w_inv.shape 

(374, 20, 9, 8)

In [12]:
w_trans = np.sum(w_inv*beta, axis=0)/np.sum(w_inv, axis=0)
w_trans.shape

(20, 9, 8)

In [13]:
np.min(w_trans)

0.0

### tau calc

In [14]:
multiplier_mat = np.ones((2,20,10,8))
for o in range(1,20):
    multiplier_mat[0,o] = multiplier_mat[0,o-1]*o_trans[o-1]
for w in range(1,10):
    multiplier_mat[1,:,w] = multiplier_mat[1,:,w-1]*w_trans[:,w-1]

tau_mat = multiplier_mat[0]*multiplier_mat[1]
print(tau_mat[6,0])
for j in range(8):
    if tau_mat[6,0,j]!=0:
        tau_mat[:,:,j] = tau_mat[:,:,j]/(tau_mat[6,0,j]) # tau_mat[6,0,:]=1
    else:
        tau_mat[:,:,j] = (tau_mat[:,:,j])/(tau_mat[6,0,j]+1.e-15) 

[3.90253334e-089 1.52017033e-060 2.75680260e-100 0.00000000e+000
 5.48149911e-097 0.00000000e+000 1.22571315e-102 8.33415803e-102]


In [15]:
# tau_mat[18]

### parameter estimation

In [16]:
# Function to calculate the log posterior
def log_posterior(p, X, a, tau, m):
    log_p = np.sum([(np.sum(X[:, :, j]) + a[j] - 1) * np.log(p[j]) for j in range(len(p))])
    log_p -= np.sum([m[o, w] * np.log(np.sum([tau[o, w, j] * p[j] for j in range(len(p))])+1.e-15) for o in range(X.shape[0]) for w in range(X.shape[1])])
    return log_p

# Metropolis within Gibbs sampler
def metropolis_within_gibbs(X, a, tau, m, num_samples, burn_in):
    num_params = len(a)
    samples = np.zeros((num_samples, num_params))
    current_sample = np.ones(num_params) / num_params  # Initial sample
    for i in range(num_samples + burn_in):
        for j in range(num_params):
            proposal = np.random.dirichlet([np.sum(X[:, :, j]) + a[j] for _ in range(num_params)])
            acceptance_ratio = np.exp(log_posterior(proposal, X, a, tau, m) - log_posterior(current_sample, X, a, tau, m))
            if np.random.rand() < acceptance_ratio:
                current_sample = proposal
        if i >= burn_in:
            samples[i - burn_in] = current_sample
    return samples

In [17]:
def met_sampling(i):
    c = 60
    a = c * np.sum(X/(tau_mat+1.e-15), axis=tuple([0,1,2])) 
    a /= np.sum(X/(tau_mat+1.e-15))
    # tau = np.random.rand(3, 4, 8) 
    m = np.sum(X[i], axis=-1)
    num_samples = 1000
    burn_in = 100
    samples = metropolis_within_gibbs(X[i], a, tau_mat, m, num_samples,burn_in)
    return np.mean(samples, axis=0)

In [18]:
## Probability matrix calc
prob = []
for i in [1,89]: # range(X.shape[0])
    param_est = met_sampling(i)
    print(param_est)
    vals = tau_mat*param_est
    prob.append(vals/(vals.sum(axis=-1, keepdims=True)+1.e-15))

[1.84685911e-01 6.58002921e-01 1.25852725e-03 4.93422938e-02
 6.26653749e-02 4.35702739e-02 4.10151945e-04 6.45453172e-05]
[0.18972788 0.37512131 0.10917511 0.06232529 0.07489923 0.03976329
 0.07777007 0.07121781]


In [21]:
prob = np.array(prob)

print(prob[:, 6, 0])

[[2.03603208e-01 7.25401872e-01 1.38743765e-03 0.00000000e+00
  6.90841618e-02 0.00000000e+00 4.52163629e-04 7.11566657e-05]
 [2.11299111e-01 4.17770958e-01 1.21587842e-01 0.00000000e+00
  8.34149433e-02 0.00000000e+00 8.66121831e-02 7.93149615e-02]]


In [ ]:
np.save("prob_mat.npy", prob)

In [ ]:
# Modifications: - weigh recent matches more (use decay ratio)